# Mineração de Regras de Associação do Dataset SAGE
_Refinamento e busca de padrões para desenvolvimento de experiências gamificadas_

Para o primeiro trabalho da disciplina de Inteligência Computacional, devemos selecionar um dataset público para mineração de dados resultando na coleta de regras de associação.

O dataset utilizado foi publicado através de uma pesquisa com o intuito de ampliar a base de dados ligadas a jogos gamificados com o propósito de analisar as preferências de usuários e como enchergam diversas características ligadas ao game design.

## Link para publicação
[SAGE: A dataset for Smart Adaptive Gamified Education](https://sol.sbc.org.br/index.php/sbie/article/view/26738/26557)

O [dataset original](https://github.com/ArmandoToda/Paper_SBIE2023) pode ser encontrado no repositório do autor no github.

# Metodologia
Criei o repositório e arquivo jupyter para executar as etapas de mineração necessárias para busca de regras de associação:

## Definição de variáveis para operação do projeto

In [152]:

import os
import pandas as pd
import requests

##Variáveis iniciais de referência
csv_filename = "SAGEDataset.csv"
# URL do arquivo XLSX no GitHub
xlsx_url = "https://github.com/ArmandoToda/Paper_SBIE2023/raw/main/DATA.xlsx"
# Nome temporário para o arquivo XLSX baixado
xlsx_filename = "DATA.xlsx"

##Filtro de dados podem ser usados para montar nichos diferentes no dataset
# Exemplo de filtro para o gênero de jogo, país de origem, gênero do jogador, idade, etc.
filterBy = {
    # 'country': ['br', 'us', 'gr', 'uk', 'cn', 'it', 'in', 'es', 'de', 'fr'],
    # 'game_genre': ['action'],
}

min_products = 1
min_support = 0.2
confidence = 0.8


## Obtenção do dataset

In [153]:
if not os.path.exists("assets/"+csv_filename):
    print(f"{csv_filename} não encontrado. Baixando e convertendo...")
    
    # Baixa o arquivo XLSX
    response = requests.get(xlsx_url)
    if response.status_code == 200:
        with open(xlsx_filename, "wb") as file:
            file.write(response.content)
        print(f"Arquivo baixado: {xlsx_filename}")
    else:
        raise Exception(f"Falha ao baixar o arquivo. Status code: {response.status_code}")
    
    """Converte um arquivo XLSX para CSV."""
    try:
        data = pd.read_excel(xlsx_filename)
        data.to_csv("assets/"+csv_filename, index=False)
        print(f"Arquivo convertido para CSV: {csv_filename}")
    except Exception as e:
        raise Exception(f"Erro ao converter XLSX para CSV: {e}")
    
    # Remove o arquivo XLSX temporário
    if os.path.exists(xlsx_filename):
        os.remove(xlsx_filename)
        print(f"Arquivo temporário removido: {xlsx_filename}")
else:
    print(f"{csv_filename} já existe. Nenhuma ação necessária.")

SAGEDataset.csv já existe. Nenhuma ação necessária.


## Preparação dos dados

In [154]:
try:
    df = pd.read_csv("assets/"+csv_filename)  # Use csv_filename em vez de file_path
    
    # Remover duplicatas
    df.drop_duplicates(inplace=True)

    # Remover linhas inválidas (com valores NaN)
    df.dropna(inplace=True)
    
    df['gender'] = df['gender'].apply(lambda x: "male" if x == "Male" else "female" if x == "Female" else "other")
    df['age_group'] = pd.cut(df['age_in_years'], bins=[0, 14, 24, 34, 44, float('inf')], 
                            labels=['<15', '15-24', '25-34', '35-44', '>44'])
    df.drop('age_in_years', axis=1, inplace=True)
    df['years_playing'] = pd.cut(df['years_playing'], bins=[0, 10, 20, 30, float('inf')], 
                                labels=['<10', '10-20', '21-30', '>30'])
    
    # print(df['country'].unique())
    
    country_mapping = {
        'Brazil': 'br',
        'United States': 'us',
        'Greece': 'gr',
        'United Kingdom': 'uk',
        'China': 'cn',
        'Italy': 'it',
        'India': 'in',
        'Spain': 'es',
        'Germany': 'de',
        'France': 'fr',
        'Turkey': 'tr',
        'South Korea': 'kr',
        'Japan': 'jp',
        'Russia': 'ru',
        'Canada': 'ca',
        'Australia': 'au',
        'Mexico': 'mx',
        'Netherlands': 'nl',
        'Argentina': 'ar',
        'Sweden': 'se',
        'Norway': 'no',
        'Finland': 'fi',
        'Denmark': 'dk',
        'Belgium': 'be',
        'Poland': 'pl',
        'Portugal': 'pt',
        'Switzerland': 'ch',
        'Austria': 'at',
        # Adicione outros países conforme necessário
    }
    df['country'] = df['country'].apply(lambda x: country_mapping[x] if x in country_mapping else 'other')
    
    df['time_per_week'] = pd.cut(df['time_per_week'], bins=[0, 5, 15, 25, 30, float('inf')], 
                                labels=['<5', '5-15', '16-25', '26-30', '>30'])
    df['game_genre'] = df['game_genre'].str.split(':').str[0].str.lower()
    
    df['game_setting'] = df['game_setting'].apply(lambda x: "multiplayer" if x.contains('Multiplayer') else "singleplayer")
    
    print("Primeiras linhas do dataset:")
    print(df.head())  # Exibe as primeiras linhas do DataFrame
    
    # Salvar o DataFrame limpo (opcional)
    cleaned_file_path = "assets/Cleaned_" + csv_filename
    df.to_csv(cleaned_file_path, index=False)
    print(f"\nDataset limpo salvo em: {cleaned_file_path}")
except Exception as e:
    print(f"Erro ao carregar o arquivo CSV: {e}")

Erro ao carregar o arquivo CSV: 'str' object has no attribute 'contains'


## Binarização de Dataset

In [155]:
from mlxtend.frequent_patterns import apriori, association_rules

columns_to_binarize = ['gender', 'age_group', 'country', 'years_playing', 'time_per_week', 'game_genre', 'game_setting']
df_binarized = pd.get_dummies(df, columns=columns_to_binarize, prefix=columns_to_binarize)
# Remover itens de columns_to_binarize que estão presentes como chaves em filterBy
columns_to_binarize = [col for col in columns_to_binarize if col not in filterBy.keys()]
# Print se algum item foi removido
if len(columns_to_binarize) != len(df.columns):
    print(f"Removendo {len(columns_to_binarize)} colunas de binarização que estão presentes em filterBy.")

# criar coluna binária somente para colunas de interesse alto
def transform_scale_to_binary(df, column):
    """
    Transforma uma coluna de escala de 1 a 5 em uma coluna binária (0 ou 1).
    """
    # Verifica se a coluna existe no DataFrame
    if column in df.columns:
        # Aplica a transformação
        df[column] = df[column].apply(lambda x: 1 if x > 3 else 0)
        # print(f"Coluna '{column}' transformada para binária.")
    else:
        print(f"Coluna '{column}' não encontrada no DataFrame.")
    
# Aplicar a transformação para as colunas desejadas
columns_to_transform = ['points', 'levels', 'cooperation', 'competition', 'renovation',
                        'progression', 'objectives', 'puzzles', 'novelty', 'chances',
                         'social_pressure', 'acknowledgment', 'stats', 'rarity',
                         'imposed_choice', 'time_pressure', 'economy', 'sensation',
                         'reputation', 'narrative', 'storytelling']  # Substitua pelos nomes das colunas que deseja transformar
for col in columns_to_transform:
    if col in df_binarized.columns:
        transform_scale_to_binary(df_binarized, col)

# Salvar o DataFrame binarizado (opcional)
binarized_file_path = "assets/Binarized_" + csv_filename
df_binarized.to_csv(binarized_file_path, index=False)
df_binarized.head()

Removendo 7 colunas de binarização que estão presentes em filterBy.


,points,levels,cooperation,competition,renovation,progression,objectives,puzzles,novelty,chances,...,game_genre_fighting,game_genre_other,game_genre_racing,game_genre_rhythm,game_genre_role playing game,game_genre_simulation,game_genre_sports,game_genre_strategy,game_setting_Multiplayer (Playing with other people),game_setting_Singleplayer (Playing solo)
0,1,1,0,1,0,1,1,0,1,0,...,False,False,False,False,False,False,False,False,False,True
1,0,0,0,1,0,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,True
2,1,1,1,1,0,1,0,1,1,0,...,False,False,False,False,False,False,False,False,True,False
3,1,1,0,0,0,1,1,1,0,1,...,False,False,False,False,False,False,False,False,False,True
4,0,0,1,0,0,0,1,0,0,0,...,False,False,False,False,False,False,False,False,False,True


## Execução do Algoritmo Apriori

Nesse processo serão definidos algumas variáveis iniciais e depois a mineração será executada.

In [156]:
frequent_itemsets = apriori(df_binarized, min_support=min_support, use_colnames=True)

# print("\nConjuntos frequentes:")
# print(frequent_itemsets)

# Gerar regras de associação
num_itemsets = df.columns.size # Número total de colunas no começo do tratamento
rules = association_rules(frequent_itemsets, num_itemsets=num_itemsets, metric="confidence", min_threshold=confidence)

#Aplicando filtro no antecedents e consequents dos resultados
rules = rules[rules['antecedents'].apply(len) + rules['consequents'].apply(len) > min_products]
# Remover regras que contenham "progression" ou "objectives" em qualquer lado
rules = rules[
    ~rules['antecedents'].apply(lambda x: 'progression' in x or 'objectives' in x) &
    ~rules['consequents'].apply(lambda x: 'progression' in x or 'objectives' in x)
]

# Converter 'antecedents' e 'consequents' de frozenset para string
rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

rules.head()

rule_file_path = "assets/rules_" + csv_filename
rules.to_csv(rule_file_path, index=False)

/Users/realima/Work/UFPA/PythonProjects/SAGEMining/.venv/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



# Plotting de Resultados
Pesquisando maneiras de visualizar os resultados de associação encontrei um artigo que apresentava de maneira intuitiva e interativa o plot do 

In [157]:
import plotly.express as px
import webbrowser

fig = px.scatter(
    rules,
    title="Regras de Associação SAGE Dataset: Mapa Confidence x Lift com calor por Support",
    hover_data=['antecedents', 'consequents'],  # Mostrar os antecedentes e consequentes ao passar o mouse
    
    x='confidence',  # Eixo X
    y='lift',   # Eixo Y
    color='support',  # Tamanho dos pontos baseado no suporte
    size='support',  # Tamanho dos pontos baseado no suporte,
)

# Exibir o gráfico
fig.write_html('plot.html')
# Caminho para o arquivo HTML gerado
plot_file_path = os.path.abspath("plot.html")

# Abrir o arquivo no navegador padrão
webbrowser.open(f"file://{plot_file_path}")

True